In [19]:
import cv2
import os

def get_last_image_index(save_directory):
    image_files = [f for f in os.listdir(save_directory) if os.path.isfile(os.path.join(save_directory, f))]
    if not image_files:
        return 0

    indexes = [int(f.split('.')[0]) for f in image_files]
    return max(indexes)

def is_black_image(frame, threshold=30):
    # Berechne die durchschnittliche Helligkeit des Bildes
    avg_brightness = int(frame.mean())

    # Wenn die durchschnittliche Helligkeit unter dem Schwellenwert liegt, ist das Bild größtenteils schwarz
    return avg_brightness < threshold

def capture_images(camera_index, num_images, save_directory):
    # Öffne die hintere Kamera
    cap = cv2.VideoCapture(camera_index)

    if not cap.isOpened():
        print("Kamera konnte nicht geöffnet werden.")
        return

    # Erstelle das Verzeichnis, falls es nicht existiert
    os.makedirs(save_directory, exist_ok=True)

    # Wir lesen das erste Bild ohne es zu speichern
    _, _ = cap.read()

    last_index = get_last_image_index(save_directory)
    start_index = last_index + 1

    for i in range(start_index - 1, start_index + num_images - 1):
        # Bilder aufnehmen
        ret, frame = cap.read()
        if not ret:
            print("Fehler beim Erfassen des Bildes.")
            break

        # Bild anzeigen
        cv2.imshow(f"Bild {i}", frame)

        # Warte 1 Sekunde zwischen den Bildern
        cv2.waitKey(1000)

        # Überprüfe, ob das Bild schwarz ist
        if not is_black_image(frame):
            # Bild speichern
            file_name = os.path.join(save_directory, f"{i}.jpg")
            cv2.imwrite(file_name, frame)

    # Kamera freigeben und alle Fenster schließen
    cap.release()
    cv2.destroyAllWindows()

In [20]:
camera_index = 0  # 0 für die Vorderkamera, 1 für die Rückkamera (falls vorhanden)
num_images = 101
save_directory = "./dataset/img"

capture_images(camera_index, num_images, save_directory)

In [ ]:
capture_images(camera_index, num_images, save_directory)